# Example of using the orm classes

Imports

In [1]:
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base
from striplog import Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [2]:
! rm ./tmp/test_orm_db.db

Create the db engine and the database structure

In [3]:
engine = create_engine('sqlite:///tmp/test_orm_db.db', echo=True)

In [4]:
Base.metadata.create_all(engine)

2020-12-18 16:29:46,357 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-18 16:29:46,364 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 16:29:46,375 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-18 16:29:46,377 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 16:29:46,381 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Boreholes")
2020-12-18 16:29:46,384 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 16:29:46,386 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Boreholes")
2020-12-18 16:29:46,387 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 16:29:46,388 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Positions")
2020-12-18 16:29:46,393 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 16:29:46,396 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Positions")
2020-12-18 16:29:46,401 INFO sqlalchemy.engine.base.Engine ()
2020-12-18

Create the objects from las files

In [5]:
lexicon = Lexicon.default()

In [6]:
borehole_dict = {'F01':'../data/test.las', 'F02':'../data/test.las'} # boreholes to insert into the db

In [7]:
int_id = 0
bh_id = 0
pos_id = 0
boreholes = []
components = []
comp_id = 0
component_dict={}
pos_dict = {}
for bh, filename in borehole_dict.items():
    interval_number = 0
    boreholes.append(BoreholeOrm(id=bh))
    with open(filename, 'r') as las3:
        strip = Striplog.from_las3(las3.read(), lexicon)
    for c in strip.components:
        if c not in component_dict.keys():
            component_dict.update({c:comp_id})
            comp_id += 1
    d ={}
    for interval in strip:
        top = PositionOrm(id=pos_id, upper=interval.top.upper, middle=interval.top.middle,  lower=interval.top.lower)
        base = PositionOrm(id=pos_id+1, upper=interval.base.upper, middle=interval.base.middle,  lower=interval.base.lower)
        d.update({int_id:{'description':interval.description, 'interval_number' : interval_number, 
                          'top': top, 'base': base
                         }})
        interval_number+=1
        int_id += 1
        pos_id += 2

    print(d)
    boreholes[bh_id].intervals_values = d
    #boreholes[bh_id].components_values = c
    bh_id += 1 
components = {v:k for k,v in component_dict.items()}

{0: {'description': 'Anhydrite', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7fc0b3f5e160>, 'base': <core.orm.PositionOrm object at 0x7fc0b3f5e278>}, 1: {'description': 'Sandstone, grey, vf-f', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7fc0b3f5e198>, 'base': <core.orm.PositionOrm object at 0x7fc0b3f5e320>}, 2: {'description': 'Anhydrite', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7fc0b3f5e358>, 'base': <core.orm.PositionOrm object at 0x7fc0b3f5e518>}, 3: {'description': 'Dolomite', 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7fc0b3f5e438>, 'base': <core.orm.PositionOrm object at 0x7fc0b3f5e400>}, 4: {'description': 'Anhydrite', 'interval_number': 4, 'top': <core.orm.PositionOrm object at 0x7fc0b3f5e550>, 'base': <core.orm.PositionOrm object at 0x7fc0b3f5e2e8>}, 5: {'description': 'Sandstone, grey, vf-f', 'interval_number': 5, 'top': <core.orm.PositionOrm object at 0x7fc0b3f5e4a8>, 'base': <core.orm.Position

Create the session

In [8]:
Session = sessionmaker(bind=engine)
session = Session()

Create the project

In [9]:
p = Project(session)
p.add_components(components)

2020-12-18 16:29:46,731 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 16:29:46,732 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-18 16:29:46,733 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 16:29:46,736 INFO sqlalchemy.engine.base.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2020-12-18 16:29:46,737 INFO sqlalchemy.engine.base.Engine ((0, 'Siltstone, grey'), (1, 'Anhydrite'), (2, 'Sandstone, vf-f, grey'), (3, 'Dolomite'), (4, 'Siltstone, red'), (5, 'Limestone'))
2020-12-18 16:29:46,738 INFO sqlalchemy.engine.base.Engine COMMIT
2020-12-18 16:29:46,746 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 16:29:46,747 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes"
2020-12-18 16:29:46,748 INFO sqlalchemy.engine.base.Engine ()


Add boreholes into the database

In [10]:
for bh in boreholes:
    p.add_borehole(bh)

2020-12-18 16:29:46,763 INFO sqlalchemy.engine.base.Engine INSERT INTO "Boreholes" (id) VALUES (?)
2020-12-18 16:29:46,766 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-18 16:29:46,769 INFO sqlalchemy.engine.base.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2020-12-18 16:29:46,770 INFO sqlalchemy.engine.base.Engine ((0, 14.3, 14.3, 14.3, 0.0, 0.0), (1, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (2, 51.24117647058824, 51.24117647058824, 51.24117647058824, 0.0, 0.0), (3, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (4, 54.81764705882354, 54.81764705882354, 54.81764705882354, 0.0, 0.0), (5, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (6, 56.55882352941177, 56.55882352941177, 56.55882352941177, 0.0, 0.0), (7, 58.86470588235295, 58.86470588235295, 58.86470588235295, 0.0, 0.0)  ... displaying 10 of 50 total bound parameter sets ...  (48, 131.47647058823532, 131.476470588

In [11]:
p.boreholes[0].id

'F01'

In [12]:
p.boreholes[1].id='F33'

In [13]:
p.boreholes[1].id

'F33'

In [14]:
p.commit()

2020-12-18 16:29:46,884 INFO sqlalchemy.engine.base.Engine UPDATE "Boreholes" SET id=? WHERE "Boreholes".id = ?
2020-12-18 16:29:46,886 INFO sqlalchemy.engine.base.Engine ('F33', 'F02')
2020-12-18 16:29:46,889 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
p.boreholes[0].intervals[0].description

2020-12-18 16:29:46,909 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-12-18 16:29:46,910 INFO sqlalchemy.engine.base.Engine SELECT "Boreholes".id AS "Boreholes_id" 
FROM "Boreholes" 
WHERE "Boreholes".id = ?
2020-12-18 16:29:46,915 INFO sqlalchemy.engine.base.Engine ('F01',)
2020-12-18 16:29:46,923 INFO sqlalchemy.engine.base.Engine SELECT "Intervals".id AS "Intervals_id", "Intervals".borehole AS "Intervals_borehole", "Intervals".interval_number AS "Intervals_interval_number", "Intervals".description AS "Intervals_description", "Intervals".top_id AS "Intervals_top_id", "Intervals".base_id AS "Intervals_base_id" 
FROM "Intervals" 
WHERE ? = "Intervals".borehole
2020-12-18 16:29:46,930 INFO sqlalchemy.engine.base.Engine ('F01',)


'Anhydrite'

In [16]:
session.close()

2020-12-18 16:29:46,940 INFO sqlalchemy.engine.base.Engine ROLLBACK
